In [ ]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import semcor
from xml.dom import minidom
import random
from nltk.corpus.reader.wordnet import Lemma

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
import spacy
nlp = spacy.load('en_core_web_sm')

lemmatizer = WordNetLemmatizer() 
stop_words_list = []

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# Ritorna il POS TAG di una parola
def get_wordnet_pos(word):
    treebank_tag = [tag for (word, tag) in nltk.pos_tag(nltk.word_tokenize(word))][0]
    if treebank_tag.startswith('J'):
        return wn.ADJ
    elif treebank_tag.startswith('V'):
        return wn.VERB
    elif treebank_tag.startswith('N'):
        return wn.NOUN
    elif treebank_tag.startswith('R'):
        return wn.ADV
    else:
        return ''

# Ritorna una lista con le stop words
def get_stop_words() :
    if len(stop_words_list) == 0 :
        f = open("stop_words_FULL.txt", "r")
        for x in f:
            stop_words_list.append(x)
    
    return stop_words_list

# Da una frase ritorna una lista con le singole parole (lemmi) rimuovendo le parole inutili (stop words..)
def get_list_of_gains_words(sentence) :
    list_words_lemma = []
    aus_list_words = nltk.word_tokenize(sentence)
    stop_words_list = get_stop_words()

    for w in aus_list_words :
        if w.lower() not in stop_words_list :
            pos_tag = get_wordnet_pos(w)
            if pos_tag != '' :
                list_words_lemma.append(lemmatizer.lemmatize(w.lower(), pos_tag))

    return list_words_lemma

# Ritorna la lista delle parole della signature di un synset : gloss + examples
def get_signature_of_synset(synset) :
    # Gloss
    signature = get_list_of_gains_words(synset.definition())
    # Examples
    for ex in synset.examples() :
        list_ex = get_list_of_gains_words(ex)
        signature.extend(list_ex)

    return signature

# Calcola il numero di elementi comuni di due liste
def overlap_lists(list1, list2) :
    common_elements = set(list1) & set(list2)
    return len(common_elements)


In [ ]:
def lesk_algorithm(word, sentence) :
    best_sense = wn.synsets(word)[0] #prende il primo synset della parola, ossia il senso più comune
    max_overlap = 0
    context = get_list_of_gains_words(sentence) # Da una frase ritorna una lista con le singole parole (lemmi) rimuovendo le parole inutili (stop words..)
    for sense in wn.synsets(word) :#itero su ogni senso/synset della parola
        #Signature
        signature_sense = get_signature_of_synset(sense) # Ritorna la lista delle parole della signature di un synset : gloss + examples
        if len(signature_sense) < 25 : 
            #Aggiungo le info degli iponomi ( il contrario di iperonimo es ALBERO (iperonimo) PINO (iponimo))
            for hypo in sense.hyponyms() :
                signature_sense.extend(get_signature_of_synset(hypo))  
              
        #Overlap
        overlap = overlap_lists(context, signature_sense)

        if overlap > max_overlap : #se overlap è maggiore del massimo overlap
            max_overlap = overlap #aggiorno l'overlap
            best_sense = sense #aggiorno il senso migliore

    return best_sense 




In [ ]:
def get_filler(subj,obj,semtypes):
  #Inserimento filler soggetto e oggetto,  lexname: The name of the lexicographer file containing this synset
  if wn.synsets(subj): # verifico che il synset non sia vuoto per evitare IndexError
    if wn.synsets(subj)[0].lexname() not in semtypes :
        #faccio diventare semtype una lista 
        lista = []
        lista.append(subj.lower())
        #semptypes ={"lexname1":[subj1], "lexname2":[subj2]}
        semtypes[wn.synsets(subj)[0].lexname()] = lista # creo la voce nel dizionario semptype con chiave il lexname e contenuto la lista
    elif subj.lower() not in semtypes[wn.synsets(subj)[0].lexname()] : 
        semtypes[wn.synsets(subj)[0].lexname()].append(subj.lower())
    #print(wn.synsets(subj)[0].lexname())

    if wn.synsets(obj)[0].lexname() not in semtypes :
        lista = []
        lista.append(obj.lower())
        semtypes[wn.synsets(obj)[0].lexname()] = lista
    elif obj.lower() not in semtypes[wn.synsets(obj)[0].lexname()] :
        semtypes[wn.synsets(obj)[0].lexname()].append(obj.lower())
    #print(wn.synsets(obj)[0].lexname())
  return semtypes

In [ ]:
def get_clusters(subj,obj,clusters,cluster_fillers,p):
  if wn.synsets(subj): # verifico che il synset non sia vuoto per evitare IndexError
    #Aggiungo cluster con i 2 semtypes alla lista generale con le frequenze
    clus = "<" + wn.synsets(subj)[0].lexname() + "," + wn.synsets(obj)[0].lexname()  + ">"
    #Conto le frequenze della coppia di lexname <soggetto,oggetto>, 
    #alla fine clusters è fatto così:  {<soggetto,oggetto>:contatore, ...}
    if clus not in clusters :
      clusters[clus] = 1 # se è la prima volta allora il numero è 1
    else :
      clusters[clus] += 1 # altrimenti incremento di 1 il contatore

    cluster_fillers.append(clus)
    cluster_fillers.append(p)
  return clusters,cluster_fillers

In [ ]:
def esercitazione3(verb, file) :
    verb_semantics = {}
    clusters = {}
    cluster_fillers = []

    #Lettura file corpus
    file1 = open(file,"r")
    for p in file1 : # per ogni riga nel file input
        #try :
            #Analisi sintattica con spacy
            sent = p
            doc=nlp(sent) #costruisce un oggetto documento di spaCy fatto da token https://spacy.io/api/doc
            #Scorre i vari token nel parser sintattico della frase
            for token in doc:
                subj = ""
                obj = ""
                # Trova il token che corrisponde al verbo cercato e ne prende subj e obj
                if verb in token.text : #se il verbo cercato è nel testo del token
                   
                    for child in token.children : #per ogni figlio sintattico del token
                    #se la relazione sintattica tra il token e il figlio è nsubj https://spacy.io/usage/linguistic-features vedere immagine in fondo
                        if child.dep_ == "nsubj" and subj == "": 
                            subj = child.text #se il padre è in relazione di soggetto e non l'ho ancora trovato lo aggiungo a subj
                        if child.dep_ == "dobj" and obj == "" :
                            obj = child.text   #se il figlio è in relazione di oggetto e non l'ho ancora trovato lo aggiungo a obj             
                    #stampo testo token e relazione sintattica del token , poi testo del parent del token e il part-of-speech dall' Universal POS tag set.
                    #print(token.text, token.dep_, token.head.text, token.head.pos_)
                    
                    #Disambiguazione Verbo
                   
                    verb_synset = lesk_algorithm(token.text, p)
                    
                    #print("----------------Synset verbo : " + str(verb_synset))
                    #print(str(verb_synset.lexname()))

                    # Creazione lista semtypes divisa per significato verbo
                    if verb_synset not in verb_semantics :
                      #crea una voce nel dizionario verb_semantics la cui chiave è verb_synset e il contenuto è un dizionario vuoto
                      #quindi verb_semantic è dizionario di dizionari
                        verb_semantics[verb_synset] = {} 
                    """
                    a={ 1: "ciao", 2: "banana"}
                    b= a[1]
                    >>>b = ciao
                    verb_semantics è un dizionario, in questo esempio la a, mentre semtypes è il contenuto associato alla chiave 
                    verb_synset nel dizionario, nel nostro esempio la b
                    """
                    semtypes = verb_semantics[verb_synset] #mettiamo il dizionario associato alla chiave verb_synset in semptypes
                    semtypes= get_filler(subj,obj,semtypes)

                    clusters,cluster_fillers= get_clusters(subj,obj,clusters,cluster_fillers,p)
                    
            #print(p)
            #print("\n")
        #except IndexError: 
          #pass # ignora errore index out of range e va avanti
    print("#Fillers divisi per senso del verbo :")
    print(verb_semantics)
    print("\n")
    print("#Cluster con frequenze :")
    print(clusters)

    print(cluster_fillers)



esercitazione3("play", "corpus Play ridotto.txt")
#esercitazione3("show", "covid 50000 infect.txt")

#Fillers divisi per senso del verbo :
{Synset('play.n.03'): {'noun.attribute': ['effectiveness'], 'noun.relation': ['part']}, Synset('turn.n.03'): {'noun.substance': ['he'], 'noun.location': ['position'], 'noun.cognition': ['it'], 'noun.act': ['game']}, Synset('play.v.01'): {'noun.substance': ['i', 'he'], 'noun.artifact': ['ball', 'drums'], 'noun.group': ['who'], 'noun.act': ['golf', 'course', 'role'], 'noun.person': ['golfers', 'president', 'sax'], 'noun.communication': ['recitals']}, Synset('play.v.03'): {'noun.group': ['who', 'band'], 'noun.act': ['golf'], 'noun.time': ['march'], 'noun.cognition': ['sorts']}, Synset('play.n.01'): {}, Synset('looseness.n.05'): {'noun.cognition': ['components'], 'noun.relation': ['part']}, Synset('toy.v.02'): {}, Synset('act.v.03'): {'noun.attribute': ['specialty'], 'noun.person': ['man']}}


#Cluster con frequenze :
{'<noun.attribute,noun.relation>': 1, '<noun.substance,noun.location>': 1, '<noun.group,noun.act>': 2, '<noun.substance,noun.artifact>':